In [1]:
from __future__ import division, print_function, unicode_literals
from sacred.observers import MongoObserver
from dae import ex
import dispy

In [5]:
cluster_nodes = ['put', 'your', 'nodes', 'here']

mongo_db = {
    'url': 'DATABASE_IP_HERE:27017',
    'db': 'binding_via_rc'
}

In [3]:
cluster = dispy.JobCluster('dae.py', nodes=cluster_nodes)

2016-01-16 17:57:08,627 - dispy - Storing fault recovery information in "_dispy_20160116175708"
INFO:dispy:Storing fault recovery information in "_dispy_20160116175708"


In [4]:
cluster.status()

ClusterStatus(nodes=[<dispy.DispyNode object at 0x7fb865659d30>], jobs_pending=0)

In [6]:
# Random Search
nr_runs_per_dataset = 100
datasets = {
    'bars': 12, 
    'corners': 5,
    'shapes': 3,
    'multi_mnist': 3,
    'mnist_shape': 2,
    'simple_superpos':2
}

jobs = []
for ds, k in datasets.items():
    for i in range(nr_runs_per_dataset):
        job = cluster.submit('-m', '{url}:{db}.random_search'.format(**mongo_db), 'with', 
                             'random_search',
                             'dataset.name={}'.format(ds),
                             'verbose=False',
                             'em.k={}'.format(k))
        jobs.append(job)
        


In [7]:
# Multi-Train Runs
multi_jobs = []
for ds, k in datasets.items():
    if ds == "simple_superpos": continue
    for i in range(nr_runs_per_dataset):
        job = cluster.submit('-m', '{url}:{db}.train_multi'.format(**mongo_db), 'with', 
                             'random_search',
                             'dataset.name={}'.format(ds),
                             'dataset.train_set=train_multi',
                             'em.e_step=max',
                             'verbose=False',
                             'em.k={}'.format(k))
        multi_jobs.append(job)

In [8]:
# MSE-Likelihood Runs
mse_jobs = []
for ds, k in datasets.items():
    for i in range(nr_runs_per_dataset):
        job = cluster.submit('-m', '{url}:{db}.mse_likelihood'.format(**mongo_db), 'with', 
                             'random_search',
                             'dataset.name={}'.format(ds),
                             'dataset.salt_n_pepper=0.3',
                             'network_spec=Fr250',
                             'verbose=False',
                             'em.k={}'.format(k))
        mse_jobs.append(job)